## **ETL - user_items**
**En este notebook realizaremos la extracción, transformación y carga (ETL) de `user_items` Al finalizar este notbook tendremos acceso a los datos de manera mas limpia**

In [3]:
# Importando Librerías
import json             # Módulo de codificador y decodificador JSON
import ast              # Módulo de Árboles de Sintaxis Abstracta
import pandas as pd     # Librería para manipular datasets
import pyarrow as pa    # Útil para trabajar con este formato y realizar operaciones de lectura y escritura de datos
import pyarrow.parquet as pq   # Útil para leer y escribir datos en formato Parquet de manera eficiente
import gzip             # Librería para comprimir y descomprimir datos
import os               # creación de directorios y comprobación de existenciav

# Extraccion de la informacion

el siguiente código carga los datos del archivo JSON línea por línea, convirtiéndolos en diccionarios de Python y almacenándolos en una lista para su posterior procesamiento.

In [2]:
# Cargo dos de lo datasets que voy a usar


user_items_path = 'C:\\Users\\ahurt\\OneDrive\\Escritorio\\Introduction to IA\Henry\\GameRecommenderX\DataSets\\australian_users_items.json'
#user_items_path = '/content/australian_users_items.json'
data_list = []
with open(user_items_path,  encoding="utf-8") as items:
    for line in items.readlines():
         data_list.append(ast.literal_eval(line))


<>:4: SyntaxWarning: invalid escape sequence '\H'
<>:4: SyntaxWarning: invalid escape sequence '\H'
C:\Users\ahurt\AppData\Local\Temp\ipykernel_7760\215710609.py:4: SyntaxWarning: invalid escape sequence '\H'
  user_items_path = 'C:\\Users\\ahurt\\OneDrive\\Escritorio\\Introduction to IA\Henry\\GameRecommenderX\DataSets\\australian_users_items.json'


In [4]:
# Genero un Dataframe con la informacio extraida
ItemsDF = pd.DataFrame(data_list)

In [5]:
ItemsDF.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [6]:
ItemsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [7]:
ItemsDF.describe()

,items_count
count,88310.000000
mean,58.353629
std,122.312095
min,0.000000
25%,3.000000
50%,26.000000
75%,73.000000
max,7762.000000


# Transformacion, desanidar los datos

El código comienza desanidando los datos de la columna ['items'] en un DataFrame utilizando el método explode(), lo que significa que se expanden las listas en esa columna para que cada elemento de la lista tenga su propia fila en el DataFrame resultante. Luego, utiliza pd.json_normalize() para normalizar los datos JSON en cada fila de la columna 'items' expandida, estableciendo el índice como el índice original del DataFrame. Finalmente, concatena horizontalmente los DataFrames resultantes de la normalización con el DataFrame original 'ItemsExplode' mediante pd.concat(), manteniendo así la relación entre los datos normalizados y los datos originales.

In [8]:
# se desanidan los datos de la columna ['items']
ItemsExplode = ItemsDF.explode(['items'])
JsonNormalized = pd.json_normalize(ItemsExplode['items']).set_index(ItemsExplode['items'].index)
ItemsDF_unnesting = pd.concat([JsonNormalized, ItemsExplode], axis=1)

In [10]:
# ELimino la columna recien desanidada
ItemsDF_unnesting = ItemsDF_unnesting.drop(columns=['items'])

Se recorre cada Columna buscando eliminar nulos

In [14]:
for column in ItemsDF_unnesting.columns.to_list():
    ItemsDF_unnesting = ItemsDF_unnesting.dropna(subset=[column])


In [22]:
#reinicio el index del dataframe
ItemsDF_unnesting.reset_index(drop=True, inplace=True)
#reviso si hay datos nulos
ItemsDF_unnesting['playtime_forever'].isnull().sum()

0

In [23]:
ItemsDF_unnesting

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


Pueden existir mas transformaciones pertinentes, pero las voy a dejar hasta aqui por el momento solo Exporto los datos

In [24]:
ruta_carpeta = "Data_Extracted"

if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

# Asigno la ruta donde quiero guardar el parquet con el nombre que va tener
ruta_guardar_parquet = "Data_Extracted/DataFrame_items.parquet"

# Transformo a una tabla el DataFrame y luego guardo como archivo Parquet
table = pa.Table.from_pandas(ItemsDF_unnesting)
pq.write_table(table, ruta_guardar_parquet)